# Базовое решение для задачи B

In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb
%matplotlib inline

Читаем входные файлы с данными

In [2]:
transactions = pd.read_csv('transactions.csv')

In [3]:
transactions['day'] = transactions['tr_datetime'].apply(lambda x: int(x.split(' ')[0]))
del transactions['tr_datetime']

In [4]:
transactions['week_num'] = transactions['day'] // 7
transactions['week_day'] = (transactions['day'] +4) % 7
transactions['month_num'] = transactions['day'] // 30
transactions['month_day'] = transactions['day'] % 30

In [5]:
transactions['log_amount'] = np.log(np.sign(transactions.amount)*transactions.amount+1)

In [6]:
transactions_train = transactions[(transactions.month_num < 14) & (transactions.amount < 0)]
transactions_fit = transactions[(transactions.month_num >= 14) & (transactions.amount < 0)]

In [7]:
positive_features = transactions[(transactions.amount > 0)].groupby(['mcc_code']).amount.sum()

In [8]:
p_df =  pd.DataFrame()
p_df['positive_sums'] =  transactions[(transactions.amount > 0)].groupby(['mcc_code']).amount.sum()
p_df['positive_means'] = transactions[(transactions.amount > 0)].groupby(['mcc_code']).amount.mean()
p_df['mcc_code'] = transactions[(transactions.amount > 0)].groupby(['mcc_code']).amount.sum().index
p_df.index = [i for i in range(p_df.shape[0])]

In [9]:
# посчитаем для трейна средние
train_sum = transactions_train.groupby(['mcc_code', 'day']).amount.sum().unstack().fillna(0)

ans = []
n = max(transactions_train['day'])
weights = np.array([(n-i+1)/n for i in range(n+1,0,-1)])
weights_exp = np.array([0.5**i for i in range(n+1,0,-1)])
weights_3 = np.array([1/(i**0.5) for i in range(n+1,0,-1)])

w_ans = []
for k in range(184):
        values = np.log(500-train_sum.iloc[k])
        avg = np.average(values, weights=weights)
        std = np.average((values-avg)**2, weights=weights)
        avg1 = np.average(values, weights=weights_exp)
        std1 = np.average((values-avg)**2, weights=weights_exp)
        avg2 = np.average(values, weights=weights_3)
        std2 = np.average((values-avg)**2, weights=weights_3)
        ans.append([std, avg, avg1, std1, avg2, std2])

std_avg_train = pd.DataFrame(data=ans, index=train_sum.index,columns=['std_all_linear','avg_all_linear', 
                                                                      'std_all_linear1','avg_all_linear1',
                                                                      'std_all_linear2','avg_all_linear2'])

std_avg_train['mcc_code']= std_avg_train.index

std_avg_train.index = np.array([i for i in range(std_avg_train.shape[0])])

/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/pandas/core/index.py:3141: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return np.sum(name == np.asarray(self.names)) > 1


In [10]:
# посчитаем для всех 
all_sum = transactions[transactions['amount']<0].groupby(['mcc_code', 'day']).amount.sum().unstack().fillna(0)

ans = []
n = max(transactions['day'])
    
weights = np.array([(n-i+1)/n for i in range(n+1,0,-1)])
weights_exp = np.array([0.5**i for i in range(n+1,0,-1)])
weights_3 = np.array([1/(i**0.5) for i in range(n+1,0,-1)])

w_ans = []
for k in range(184):
        values = np.log(500-all_sum.iloc[k])
        avg = np.average(values, weights=weights)
        std = np.average((values-avg)**2, weights=weights)
        avg1 = np.average(values, weights=weights_exp)
        std1 = np.average((values-avg)**2, weights=weights_exp)
        avg2 = np.average(values, weights=weights_3)
        std2 = np.average((values-avg)**2, weights=weights_3)
        ans.append([std, avg, avg1, std1, avg2, std2])

std_avg_all = pd.DataFrame(data=ans, index=all_sum.index,columns=['std_all_linear','avg_all_linear', 
                                                                      'std_all_linear1','avg_all_linear1',
                                                                      'std_all_linear2','avg_all_linear2'])

std_avg_all['mcc_code']= std_avg_all.index

std_avg_all.index = np.array([i for i in range(std_avg_all.shape[0])])

/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/pandas/core/index.py:3141: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return np.sum(name == np.asarray(self.names)) > 1


In [11]:
train_sum_wd = transactions_train.groupby(['mcc_code', 'week_day', 'day']).amount.sum()
n = max(transactions_train['day'])
d = 1
weights = np.array([ ((n//7-i//7+1)/(n//7))**d for i in range(n+1,0,-1)])
weights_exp = np.array([0.5**(i//7) for i in range(n+1,0,-1)])
weights_3 = np.array([1/((i//7+1)) for i in range(n+1,0,-1)])



# train_sum_wd = train_sum_wd.reset_index()
train_sum_wd = train_sum_wd.unstack().fillna(0).reset_index()

c = train_sum_wd.columns[2:]
values = np.log(500 - train_sum_wd[c].values)
avg_weekday = np.average(values, axis=1, weights=weights)
std_weekday = np.average(np.apply_along_axis(lambda x: (x - avg_weekday)**2, 0, values),axis=1,weights=weights)

avg_weekday1 = np.average(values, axis=1, weights=weights_exp)
std_weekday1 = np.average(np.apply_along_axis(lambda x: (x - avg_weekday1)**2, 0, values),axis=1,weights=weights_exp)

avg_weekday2 = np.average(values, axis=1, weights=weights_3)
std_weekday2 = np.average(np.apply_along_axis(lambda x: (x - avg_weekday2)**2, 0, values),axis=1,weights=weights_3)

train_sum_wd['avg_wd'] = avg_weekday
train_sum_wd['std_wd'] = std_weekday

train_sum_wd['avg_wd1'] = avg_weekday1
train_sum_wd['std_wd1'] = std_weekday1

train_sum_wd['avg_wd2'] = avg_weekday2
train_sum_wd['std_wd2'] = std_weekday2

train_sum_wd = train_sum_wd[['mcc_code', 'week_day', 'avg_wd', 'std_wd']]
# train_sum_wd = train_sum_wd[['mcc_code', 'week_day', 'avg_wd', 'std_wd', 'avg_wd1', 'std_wd1', 'avg_wd2', 'std_wd2']]

/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/pandas/core/index.py:3141: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return np.sum(name == np.asarray(self.names)) > 1


In [12]:
n = max(transactions['day'])
all_sum_wd = transactions[transactions['amount']<0].groupby(['mcc_code', 'week_day', 'day']).amount.sum().unstack().fillna(0).reset_index()
d = 1
weights = np.array([ ((n//7-i//7+1)/(n//7))**d for i in range(n+1,0,-1)])
weights_exp = np.array([0.5**(i//7) for i in range(n+1,0,-1)])
weights_3 = np.array([1/((i//7+1)) for i in range(n+1,0,-1)])

c = all_sum_wd.columns[2:]
values = np.log(500 - all_sum_wd[c].values)
avg_weekday = np.average(values, axis=1, weights=weights)
std_weekday = np.average(np.apply_along_axis(lambda x: (x - avg_weekday)**2, 0, values),axis=1,weights=weights)

avg_weekday1 = np.average(values, axis=1, weights=weights_exp)
std_weekday1 = np.average(np.apply_along_axis(lambda x: (x - avg_weekday1)**2, 0, values),axis=1,weights=weights_exp)

avg_weekday2 = np.average(values, axis=1, weights=weights_3)
std_weekday2 = np.average(np.apply_along_axis(lambda x: (x - avg_weekday2)**2, 0, values),axis=1,weights=weights_3)

all_sum_wd['avg_wd'] = avg_weekday
all_sum_wd['std_wd'] = std_weekday

all_sum_wd['avg_wd1'] = avg_weekday1
all_sum_wd['std_wd1'] = std_weekday1

all_sum_wd['avg_wd2'] = avg_weekday2
all_sum_wd['std_wd2'] = std_weekday2

all_sum_wd = all_sum_wd[['mcc_code', 'week_day', 'avg_wd', 'std_wd']]
# all_sum_wd = all_sum_wd[['mcc_code', 'week_day', 'avg_wd', 'std_wd', 'avg_wd1', 'std_wd1', 'avg_wd2', 'std_wd2']]

/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/pandas/core/index.py:3141: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return np.sum(name == np.asarray(self.names)) > 1


In [13]:
dollars = pd.read_csv('USDRUB.csv', delimiter=';')
dollars["<DATE>"] = pd.to_datetime(dollars['<DATE>'], format='%Y%m%d')
dollars["day"] = [i.days for i in (dollars["<DATE>"] - dollars.iloc[0]["<DATE>"])]
dollars = dollars[["day","<OPEN>"]]
dollars.columns = ['day', 'usd']

In [14]:
holidays = ['20141101', '20141102', '20141103', '20141104', '20150101', '20150102', '20150103'
           , '20150104', '20150105', '20150106', '20150107', '20150108', '20150109', '20150110', '20150111'
           ,  '20150221', '20150222', '20150223', '20150307','20150308','20150309'
           , '20150501', '20150502', '20150503', '20150504'
           , '20150509', '20150510', '20150511'
           , '20150512', '20150513', '20150514'
           , '20151004'
           ]
first_inholidays = ['20141101', '20150101', '20150221', '20150307', '20150308', '20150501', '20150509',
                   '20150612', '20151004', '20151007', '20151014', '20151021', '20151028']
pred_holiday = ['20141031', '20141231', '20150306', '20150220', '20150508', '20150611', '20151003']

holidays = pd.DataFrame(data=holidays,columns=['day'])

default_holidays = transactions[transactions['week_day']>=5].groupby('day').size().index.values

holidays['day'] = (pd.to_datetime(holidays['day'], format='%Y%m%d') - pd.to_datetime('20140801', format='%Y%m%d'))
holidays['day'] = [i.days for i in holidays['day']]
holidays['hol'] = [1 for i in range(holidays.shape[0])]



fholidays = pd.DataFrame(data=first_inholidays,columns=['day'])

fholidays['day'] = (pd.to_datetime(fholidays['day'], format='%Y%m%d') - pd.to_datetime('20140801', format='%Y%m%d'))
fholidays['day'] = [i.days for i in fholidays['day']]
fholidays['first'] = [1 for i in range(fholidays.shape[0])]

holidays = pd.merge(holidays, fholidays, on=['day'], how='left').fillna(0)


default_holidays = transactions[transactions['week_day']>=5].groupby('day').size().index.values
default_first = transactions[transactions['week_day']==5].groupby('day').size().index.values
default_holidays = pd.DataFrame(data=default_holidays,columns=['day'])
default_holidays['hol'] = [1 for i in range(default_holidays.shape[0])]
default_first = pd.DataFrame(data=default_first,columns=['day'])
default_holidays['first'] = [1 for i in range(default_holidays.shape[0])]
default_holidays = pd.merge(default_holidays, default_first, on=['day'], how='left').fillna(0)
holidays = pd.concat([holidays, default_holidays])


holidays.index = [i for i in range (holidays.shape[0])]


In [15]:
month_means = transactions_train[transactions_train.month_num.isin([11, 12, 13])].groupby(['mcc_code', 'month_num']).amount.mean().unstack().fillna(0).reset_index()
month_means.columns = ['mcc_code'] + ['mean_month_' + str(x) for x in month_means.columns[1:]]

/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/pandas/core/index.py:3141: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return np.sum(name == np.asarray(self.names)) > 1


In [16]:
month_means_all = transactions[(transactions['amount']<0) & (transactions.month_num.isin([12, 13, 14]))].groupby(['mcc_code', 'month_num']).amount.mean().unstack().fillna(0).reset_index()
month_means_all.columns = ['mcc_code'] + ['mean_month_' + str(int(x)-1) for x in month_means_all.columns[1:]]

/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/pandas/core/index.py:3141: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return np.sum(name == np.asarray(self.names)) > 1


Берём расходные транзакции и формируем тестовую выборку

In [17]:
# fit model
train_transactions = transactions_train.copy() #transactions[transactions.amount < 0].copy()

test_transactions = pd.DataFrame(columns=train_transactions.mcc_code.unique(), 
                             index=np.arange(1, 38) + train_transactions.day.max())
test_transactions = test_transactions.unstack().reset_index().dropna(axis=1)
test_transactions.columns = ['mcc_code', 'day']
train_grid = pd.DataFrame(columns=train_transactions.mcc_code.unique(), 
                      index=train_transactions.day.unique())
train_grid = train_grid.unstack().reset_index().dropna(axis=1)
train_grid.columns = ['mcc_code', 'day']

for tr_table in [test_transactions, train_grid]:
    tr_table['week_num'] = tr_table['day'] // 7
    tr_table['week_day'] = (tr_table['day']+4) % 7
    tr_table['month_num'] = tr_table['day'] // 30
    tr_table['month_day'] = tr_table['day'] % 30

train_transactions = \
pd.merge(train_grid,
         train_transactions.groupby(['day', 'week_num', 'week_day', 'month_num', 'month_day', 'mcc_code'])[['amount']]\
             .sum().reset_index(),
         how='left').fillna(0)

for day_shift in [-1, 0, 1]:
    for month_shift in train_transactions.month_num.unique()[1:]:
        train_shift = train_transactions.copy()
        train_shift['month_num'] += month_shift
        train_shift['month_day'] += day_shift
        train_shift['amount_day_{}_{}'.format(day_shift, month_shift)] = np.log(-train_shift['amount'] + 1)
        train_shift = train_shift[['month_num', 'month_day', 'mcc_code', 'amount_day_{}_{}'.format(day_shift, month_shift)]]

        train_transactions = pd.merge(train_transactions, train_shift, 
                                      on=['month_num', 'month_day', 'mcc_code'], how='left').fillna(0)
        test_transactions = pd.merge(test_transactions, train_shift, 
                                     on=['month_num', 'month_day', 'mcc_code'], how='left').fillna(0)

train_transactions = pd.merge(train_transactions, std_avg_train, on=['mcc_code'], how='left')

test_transactions = pd.merge(test_transactions, std_avg_train, on=['mcc_code'], how='left')

train_transactions = pd.merge(train_transactions, train_sum_wd, on=['mcc_code', 'week_day'], how='left')

test_transactions = pd.merge(test_transactions, train_sum_wd, on=['mcc_code', 'week_day'], how='left')

train_transactions = pd.merge(train_transactions, dollars, on=['day'], how='left')

test_transactions = pd.merge(test_transactions, dollars, on=['day'], how='left')


train_transactions = pd.merge(train_transactions, month_means, on=['mcc_code'], how='left')

test_transactions = pd.merge(test_transactions, month_means, on=['mcc_code'], how='left')



test_transactions = pd.merge(test_transactions, transactions_fit.groupby(['mcc_code', 'day']).amount.sum().reset_index()
                             , on=['mcc_code', 'day'], how='left')

test_transactions = test_transactions.fillna(0)

train_transactions = pd.merge(train_transactions, p_df, on=['mcc_code'], how='left')

test_transactions = pd.merge(test_transactions, p_df, on=['mcc_code'], how='left')

train_transactions = pd.merge(train_transactions, holidays, on=['day'], how='left')

test_transactions = pd.merge(test_transactions, holidays, on=['day'], how='left')





shift = 500
train = pd.get_dummies(train_transactions, columns=['mcc_code']).fillna(0)
train_fit = pd.get_dummies(test_transactions, columns=['mcc_code']).fillna(0)
clmn = train.columns.difference(['amount', 'log_amount'])

In [19]:
clf2 = xgb.XGBRegressor(max_depth=4, learning_rate=0.05, n_estimators=801, objective='reg:linear')

clf2.fit(train[clmn], np.log(-train['amount'] + shift), early_stopping_rounds=40, eval_metric="rmse", eval_set=[(train_fit[clmn], np.log(-train_fit['amount'] + shift))])

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.05, max_delta_step=0, max_depth=4,
       min_child_weight=1, missing=None, n_estimators=801, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [20]:
from sklearn.ensemble import RandomForestRegressor
num_trees = 1000
model = RandomForestRegressor(n_estimators=num_trees, n_jobs=-1)
model.fit(train[clmn].fillna(0), np.log(-train['amount'] + shift))

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=1000, n_jobs=-1, oob_score=False,
           random_state=None, verbose=0, warm_start=False)

In [21]:
from sklearn.linear_model import LinearRegression, Ridge
linear_clf = LinearRegression()
linear_clf.fit(train[clmn], np.log(-train['amount'] + shift))

/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/scipy/linalg/basic.py:884: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [27]:
from sklearn.metrics import mean_squared_error

lr_p = linear_clf.predict(train_fit[clmn])
lr_p[(np.e**lr_p - shift) < 0] = 2*np.log(2)+3*np.log(5)
xgb_p = clf2.predict(train_fit[clmn])
rf_p = model.predict(train_fit[clmn])
print("linear = ",mean_squared_error(lr_p, np.log(-train_fit['amount'] + shift))**0.5)
print("xgb = ",mean_squared_error(xgb_p, np.log(-train_fit['amount'] + shift))**0.5)
print("rf = ",mean_squared_error(rf_p, np.log(-train_fit['amount'] + shift))**0.5)



linear =  1.60298097918
xgb =  1.56339514092
rf =  1.60663070556


In [41]:
min_err = 3
a_min, b_min, c_min = 0, 1, 0
step = 200
for  a  in range(step):
    for b in range(step-a):
        c = step - b - a
        mixed_ans = xgb_p/step*a+ rf_p/step*b + lr_p/step*c
        mixed_ans[(np.e**mixed_ans - shift)<0] = 0
        err = mean_squared_error(mixed_ans,  np.log(-train_fit['amount'] + shift))**0.5
        if err < min_err:
            a_min, b_min, c_min = a/step, b/step, c/step
            min_err = err
print(min_err)
(a_min, b_min, c_min)           

1.56189418425


(0.84, 0.155, 0.005)

теперь строим трейны на всех данных и предиктим

In [32]:
train_transactions = transactions[transactions.amount < 0].copy()

test_transactions = pd.DataFrame(columns=train_transactions.mcc_code.unique(), 
                             index=np.arange(1, 31) + train_transactions.day.max())
test_transactions = test_transactions.unstack().reset_index().dropna(axis=1)
test_transactions.columns = ['mcc_code', 'day']
train_grid = pd.DataFrame(columns=train_transactions.mcc_code.unique(), 
                      index=train_transactions.day.unique())
train_grid = train_grid.unstack().reset_index().dropna(axis=1)
train_grid.columns = ['mcc_code', 'day']

for tr_table in [test_transactions, train_grid]:
    tr_table['week_num'] = tr_table['day'] // 7
    tr_table['week_day'] = (tr_table['day']+4) % 7
    tr_table['month_num'] = tr_table['day'] // 30
    tr_table['month_day'] = tr_table['day'] % 30

train_transactions = \
pd.merge(train_grid,
         train_transactions.groupby(['day', 'week_num', 'week_day', 'month_num', 'month_day', 'mcc_code'])[['amount']]\
             .sum().reset_index(),
         how='left').fillna(0)
for day_shift in [-1, 0, 1]:
    for month_shift in train_transactions.month_num.unique()[1:]:
        train_shift = train_transactions.copy()
        train_shift['month_num'] += month_shift
        train_shift['month_day'] += day_shift
        train_shift['amount_day_{}_{}'.format(day_shift, month_shift)] = np.log(-train_shift['amount'] + 1)
        train_shift = train_shift[['month_num', 'month_day', 'mcc_code', 'amount_day_{}_{}'.format(day_shift, month_shift)]]

        train_transactions = pd.merge(train_transactions, train_shift, 
                                      on=['month_num', 'month_day', 'mcc_code'], how='left').fillna(0)
        test_transactions = pd.merge(test_transactions, train_shift, 
                                     on=['month_num', 'month_day', 'mcc_code'], how='left').fillna(0)

train_transactions = pd.merge(train_transactions, std_avg_all, on=['mcc_code'], how='left')

test_transactions = pd.merge(test_transactions, std_avg_all, on=['mcc_code'], how='left')

train_transactions = pd.merge(train_transactions, dollars, on=['day'], how='left')

test_transactions = pd.merge(test_transactions, dollars, on=['day'], how='left')

train_transactions = pd.merge(train_transactions, all_sum_wd, on=['mcc_code', 'week_day'], how='left')

test_transactions = pd.merge(test_transactions, all_sum_wd, on=['mcc_code', 'week_day'], how='left')

train_transactions = pd.merge(train_transactions, month_means_all, on=['mcc_code'], how='left')

test_transactions = pd.merge(test_transactions, month_means_all, on=['mcc_code'], how='left')

train_transactions = pd.merge(train_transactions, p_df, on=['mcc_code'], how='left')

test_transactions = pd.merge(test_transactions, p_df, on=['mcc_code'], how='left')

train_transactions = pd.merge(train_transactions, holidays, on=['day'], how='left')

test_transactions = pd.merge(test_transactions, holidays, on=['day'], how='left')

train = pd.get_dummies(train_transactions, columns=['mcc_code'])
train = train.fillna(0)

test = pd.get_dummies(test_transactions, columns=['mcc_code'])
test = test.fillna(0)

In [33]:
xgb_ans = clf2.predict(test[clmn])
rf_ans = model.predict(test[clmn])
linear_ans = linear_clf.predict(test[clmn])

In [36]:
mixed = a_min*xgb_ans + b_min*rf_ans + c_min*linear_ans
mixed = np.e**mixed - shift
mixed[mixed<0] = 0

In [38]:
test_transactions['volume'] = mixed

Предсказываем объём трат для тестовых данных и создаём файл с ответом

In [39]:
test_transactions[['mcc_code', 'day', 'volume']].to_csv('b.csv', index=False)